<a href="https://colab.research.google.com/github/rajni-arora/Deep-Learning-Projects/blob/main/Sentiment_analysis_Hugging_face_transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 2.6 MB 4.3 MB/s 
     |████████████████████████████████| 895 kB 43.7 MB/s 
     |████████████████████████████████| 636 kB 32.3 MB/s 
     |████████████████████████████████| 3.3 MB 41.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [3]:
from transformers import BertForSequenceClassification

1. First we need to decide which model t use.
2. Bcoz the Transformers library allows us to tape, retrain models and use those 
3. We can either use them roll and fine tune them for our own use case
4. Search model on official website on Hugging Face



In [4]:
model_name  = 'ProsusAI/finbert'

In [5]:
model = BertForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [6]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

1. Tokenize
2. Token IDs -> model
3. Model activation -> probablities(using Softmax)
4. Argmax of those probs

In [7]:
# this is our example text
txt = ("Given the recent downturn in stocks especially in tech which is likely to persist as yields keep going up, "
       "I thought it would be prudent to share the risks of investing in ARK ETFs, written up very nicely by "
       "[The Bear Cave](https://thebearcave.substack.com/p/special-edition-will-ark-invest-blow). The risks comes "
       "primarily from ARK's illiquid and very large holdings in small cap companies. ARK is forced to sell its "
       "holdings whenever its liquid ETF gets hit with outflows as is especially the case in market downturns. "
       "This could force very painful liquidations at unfavorable prices and the ensuing crash goes into a "
       "positive feedback loop leading into a death spiral enticing even more outflows and predatory shorts.")

tokens = tokenizer.encode_plus(txt, max_length=512, truncation=True, padding='max_length',
                               add_special_tokens=True, return_tensors='pt')

tokens

{'input_ids': tensor([[  101,  2445,  1996,  3522,  2091, 22299,  1999, 15768,  2926,  1999,
          6627,  2029,  2003,  3497,  2000, 29486,  2004, 16189,  2562,  2183,
          2039,  1010,  1045,  2245,  2009,  2052,  2022, 10975, 12672,  3372,
          2000,  3745,  1996, 10831,  1997, 19920,  1999, 15745,  3802, 10343,
          1010,  2517,  2039,  2200, 19957,  2011,  1031,  1996,  4562,  5430,
          1033,  1006, 16770,  1024,  1013,  1013,  1996,  4783,  2906, 27454,
          1012,  4942,  9153,  3600,  1012,  4012,  1013,  1052,  1013,  2569,
          1011,  3179,  1011,  2097,  1011, 15745,  1011, 15697,  1011,  6271,
          1007,  1012,  1996, 10831,  3310,  3952,  2013, 15745,  1005,  1055,
          5665, 18515, 21272,  1998,  2200,  2312,  9583,  1999,  2235,  6178,
          3316,  1012, 15745,  2003,  3140,  2000,  5271,  2049,  9583,  7188,
          2049,  6381,  3802,  2546,  4152,  2718,  2007,  2041, 12314,  2015,
          2004,  2003,  2926,  1996,  

max_length - this tell the tokenizer the maximum number of tokens we want to see in each sample, for BERT we almost always use 512 as that is the length of sequences that BERT consumes.

truncation - if our input string txt contains more tokens than allowed (specified in max_length parameter) then we cut all tokens past the max_length limit.

padding - if our input string txt contains less tokens than specified by max_length then we pad the sequence with zeros (0 is the token ID for '[PAD]' - BERTs padding token).

add_special_tokens - whether or not to add special tokens, when using BERT we always want this to be True unless we are adding them ourselves.


1. [PAD]- 0 - Used to fill empty space when input sequence is shorter than required sequence size for model

2. [UNK] - 100	- If a word/character is not found in BERTs vocabulary it will be represented by this unknown token

3. [CLS]	- 101	- Represents the start of a sequence
[SEP]	102	Seperator token to denote the end of a sequence 
and as a seperator where there are multiple sequences

4. [MASK]	- 103	- Token used for masking other tokens, used for masked language modeling
Note that our tokenized sequence begins with 101, the seperator token 102 can be found seperating the input sequence and padding tokens 0.


 without **kwargs
random_func(var1='hello', var2='world')

 with **kwargs
input_dict = {'var1': 'hello', 'var2': 'world'}
random_func(**input_dict)

In [8]:
output = model(**tokens)

output

SequenceClassifierOutput([('logits',
                           tensor([[-1.8200,  2.4484,  0.0216]], grad_fn=<AddmmBackward>))])

You will notice here that the output logits tensor cannot be a set of probabilities because probability values must be within the range 0-1. These are infact the final output activations from BERT, to convert these into probabilities we must apply a softmax function. We will be using the PyTorch implementation of softmax for this, which we import from torch.nn.functional.

In [9]:
import torch.nn.functional as F

# apply softmax to the logits output tensor of our model (in index 0) across dimension -1
probs = F.softmax(output[0], dim=-1)

probs

tensor([[0.0127, 0.9072, 0.0801]], grad_fn=<SoftmaxBackward>)

In [10]:
import torch

pred = torch.argmax(probs)

pred

tensor(1)

(We use dim=-1 as -1 signifies our tensors final dimension, so if we had a 3D tensor with dims [0, 1, 2] writing dim=-1 is the equivalent to writing dim=2. In this case if we wrote dim=-2 this would be the equivalent to writing dim=1. For a 2D tensor with dims [0, 1], dim=-1 is the equivalent of dim=1.)

Now we have a tensor containing three classes, all with outputs within the probability range of 0-1, these are our probabilities! We can see that class index 1 has the highest probability with a value of 0.9072. We can use PyTorch's argmax function to extract this, we can use argmax after importing torch.

In [11]:
pred.item()

1